# Data Preparation

In [132]:
import os
import pandas as pd
from pandas.api.types import CategoricalDtype

In [133]:
def create_dataframe_from_races_csvs_multi_years(base_path, years):
    required_columns = ['Pos', 'Car #', 'Class', 'Drivers', 'Team', 'Car', 'Time', 'Laps', 'Gap']
    all_data = []  # List to hold all data rows

    for year in years:  # Process each year in the list
        year_path = os.path.join(base_path, str(year))  # Path to the specific year

        if not os.path.exists(year_path):
            print(f"The directory for year {year} does not exist.")
            continue  # Skip to the next year if the directory doesn't exist

        for meeting in os.listdir(year_path):  # Iterate through all meetings in the year
            meeting_path = os.path.join(year_path, meeting)
            races_path = os.path.join(meeting_path, "Races")  # Path to the 'Races' folder

            if os.path.exists(races_path):
                for race_file in os.listdir(races_path):  # Iterate through all race files
                    file_path = os.path.join(races_path, race_file)
                    try:
                        df = pd.read_csv(file_path)
                        if set(required_columns).issubset(df.columns):
                            df['Season'] = year
                            df['Meeting'] = meeting.replace("_", " ")
                            df['Race name'] = race_file.replace(".csv", "").replace("_", " ")
                            all_data.append(df[["Season", "Meeting", "Race name"] + required_columns])
                        else:
                            print(f"Skipping {file_path} due to missing required columns.")
                    except Exception as e:
                        print(f"Error reading {file_path}: {e}")

    # Concatenate all data into a single DataFrame
    final_df = pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()
    return final_df


## Load the Data About Races 

In [134]:
base_path = ".\data_csv"  # Update to your path
years = [2021, 2022, 2023]
all_races = create_dataframe_from_races_csvs_multi_years(base_path, years)

## Exploratory Data Analysis (EDA)

**`Disclaimer:`** <br>
`I haven't found any information that explains what data these columns actually contain.`<br>
`Next will be my guesses about the true meaning of the columns.`<br><br>


The **raw** dataset contains information about various races, with the following columns:

1. **Season**: The year the race took place;
2. **Meeting**: The location of the meeting;
3. **Race name**: The name of the race;
4. **Pos**: The finishing position in the race;
5. **Car #**: The car number;
6. **Class**: The racing class in which a team competes in a race;
7. **Drivers**: The names of the drivers;
8. **Team**: The name of the team;
9. **Car**: The make and model of the car;
10. **Time**: Best lap time;
11. **Laps**: The number of laps completed;
12. **Gap**: Represent the time difference between each car and the car ahead of it, with a slight twist: <br>
    * If a car is on the same lap as the leader of the race, the Gap shows the actual time difference. <br>
    * If a car is one or more laps behind, the Gap resets and shows the time difference to the car ahead within the same lap, not the overall race leader.


In [135]:
all_races.head()

,Season,Meeting,Race name,Pos,Car #,Class,Drivers,Team,Car,Time,Laps,Gap
0,2021,Barcelona,Main Race,1,88,Pro Cup,"Raffaele Marciello, Felipe Fraga, Jules Gounon",AKKA ASP,Mercedes-AMG GT3,1:47.211,95.0,NaN
1,2021,Barcelona,Main Race,2,54,Pro Cup,"Klaus Bachler, Christian Engelhart, Matteo Cai...",Dinamic Motorsport,Porsche 911 GT3-R (991.II),1:47.148,95.0,2.174
2,2021,Barcelona,Main Race,3,32,Pro Cup,"Dries Vanthoor, Robin Frijns, Charles Weerts",Team WRT,Audi R8 LMS GT3,1:47.612,95.0,4.036
3,2021,Barcelona,Main Race,4,63,Pro Cup,"Mirko Bortolotti, Marco Mapelli, Andrea Caldar...",Orange 1 FFF Racing Team,Lamborghini Huracan GT3 Evo,1:47.027,95.0,9.511
4,2021,Barcelona,Main Race,5,4,Pro Cup,"Maro Engel, Luca Stolz, Nico Bastian",HRT,Mercedes-AMG GT3,1:47.588,95.0,9.984


### Data Types and Missing Values

Data Types should be:

1. **Season**: is a numerical and discrete variable, it should be represented as **integer**;
2. **Meeting**: is categorical and nominal variable, it should be represented as **object**;
3. **Race name**: is categorical and nominal variable, it should be represented as **object**;
4. **Pos**: is categorical and ordinal variable, it should be represented as **integer**;
5. **Car #**: is categorical and discrete variable, it should be represented as **integer**;
6. **Class**: is categorical and ordinal variable, it should be represented as **category**;
7. **Drivers**: is categorical and nominal variable, it should be represented as **object**;
8. **Team**: is categorical and nominal variable, it should be represented as **object**;
9. **Car**: is categorical and nominal variable, it should be represented as **object**;
10. **Time**: is numerical and continuous variable, it should be represented as **timedelta**;
11. **Laps**: is a numerical and discrete variable, it should be represented as **integer**;
12. **Gap**: is numerical and continuous variable, it should be represented as **timedelta**;

In [136]:
all_races.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7606 entries, 0 to 7605
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Season     7606 non-null   int64  
 1   Meeting    7606 non-null   object 
 2   Race name  7606 non-null   object 
 3   Pos        7606 non-null   object 
 4   Car #      7606 non-null   int64  
 5   Class      7606 non-null   object 
 6   Drivers    7606 non-null   object 
 7   Team       7606 non-null   object 
 8   Car        7606 non-null   object 
 9   Time       7467 non-null   object 
 10  Laps       7533 non-null   float64
 11  Gap        7389 non-null   object 
dtypes: float64(1), int64(2), object(9)
memory usage: 713.2+ KB


##### Columns data types that differ from the required data type

4.   Pos        7606 non-null   object, should be int;
6.   Class      7606 non-null   object, should be category;
10.   Time       7467 non-null   object, should be timedelta;
11.  Laps       7533 non-null   float64, should be integer;
12.  Gap        7389 non-null   object, should be timedelta.


##### Missing Values

Columns that contain missing values:

10.   Time       7467 non-null   object; 
11.  Laps       7533 non-null   float64;
12.  Gap        7389 non-null   object.



### Inspecting and fix columns

#### 1. Pos
Column `'Pos'` is object, it's mean that most likely it contains non digit values. Then inspect column and select all not digit values.

In [137]:
print(all_races[all_races['Pos'].str.contains('\D', regex=True, na=False)]['Pos'].value_counts().reset_index())

   Pos  count
0   NC    286
1  DSQ      1


Next delete all row that contain 'NC' (Not Classified) and 'DSQ' (Disqualified).

'Not Classified': This is a term used to refer to competitors who do not meet certain criteria for classification in the final results of a race or event.

In [138]:
all_races = all_races.loc[~all_races['Pos'].isin(['NC', 'DSQ'])]
all_races.reset_index(drop=True, inplace=True)

Change 'Pos' datatype from object to int

In [139]:
all_races.loc[:, 'Pos'] = all_races['Pos'].astype('int64')

#### 2. Class
Column `'Class'` is object, but we will change it to category.

In [140]:
print(all_races['Class'].value_counts().reset_index())

          Class  count
0       Pro Cup   2614
1    Silver Cup   1870
2    Pro-AM Cup   1042
3      Gold Cup    827
4    Bronze Cup    823
5    Pro-Am Cup     72
6        AM Cup     44
7        Bronze     21
8  Invitational      6


Change 'Pro-AM Cup' to 'Pro-Am Cup'. 

And 'AM Cup' change to 'Am Cup' for the sake of uniformity. 

And 'Bronze ' change to 'Bronze Cup' according to racing compliance.

In [141]:
all_races['Class'] = all_races['Class'].replace({'Pro-AM Cup': 'Pro-Am Cup', 'AM Cup': 'Am Cup', 'Bronze': 'Bronze Cup'})

print(all_races['Class'].value_counts().reset_index())

          Class  count
0       Pro Cup   2614
1    Silver Cup   1870
2    Pro-Am Cup   1114
3    Bronze Cup    844
4      Gold Cup    827
5        Am Cup     44
6  Invitational      6


Determining the order of categories and changing data type.

In [148]:
# Determine the order for the main categories
cat_type = CategoricalDtype(categories=[
    'Pro Cup', 'Gold Cup', 'Silver Cup', 'Pro-Am Cup', 'Bronze Cup', 'Am Cup', 'Invitational'
], ordered=True)
# Change the data type of the 'Class' column
all_races['Class'] = all_races['Class'].astype(cat_type)

all_races.insert(loc=6, column='Special Class', value=False)
# Indicate that 'Invitational' is a special class
all_races.loc[all_races['Class'] == 'Invitational', 'Special Class'] = True

In [158]:
print(all_races[all_races['Special Class'] == True]['Class'])

8      Invitational
15     Invitational
52     Invitational
58     Invitational
98     Invitational
105    Invitational
Name: Class, dtype: category
Categories (7, object): ['Pro Cup' < 'Gold Cup' < 'Silver Cup' < 'Pro-Am Cup' < 'Bronze Cup' < 'Am Cup' < 'Invitational']


all_races

#### 1. Season

In [145]:
print(all_races['Season'].unique())

[2021 2022 2023]


#### 2. Meeting

In [146]:
print(all_races['Meeting'].unique())

['Barcelona' 'Brands Hatch' 'Circuit Paul Ricard 1000Km' 'Magny-Cours'
 'Misano' 'Monza' 'Nürburgring' 'TotalEnergies 24 Hours of Spa' 'Valencia'
 'Zandvoort' 'Hockenheim' 'Imola' 'CrowdStrike 24 Hours of Spa']


In [147]:
print(all_races['Class'].value_counts().reset_index())


          Class  count
0       Pro Cup   2614
1    Silver Cup   1870
2    Pro-Am Cup   1114
3    Bronze Cup    844
4      Gold Cup    827
5        Am Cup     44
6  Invitational      6
